### 让实例的方法成为类的方法

In [1]:
class Kls1:
    bar = 1
    def foo(self):
        print('in foo')
    @classmethod
    def class_foo(cls):
        print(cls.bar)
        print(cls.__name__)
        cls().foo()

In [2]:
Kls1.class_foo()

1
Kls1
in foo


#### 类或对象的绑定方法：

In [6]:
Kls1.class_foo

<bound method Kls1.class_foo of <class '__main__.Kls1'>>

In [5]:
k = Kls1()
k.class_foo

<bound method Kls1.class_foo of <class '__main__.Kls1'>>

In [7]:
k.foo

<bound method Kls1.foo of <__main__.Kls1 object at 0x107228f10>>

#### 实例也可以调用类方法：

In [8]:
class Story:
    snake = 'python'
    def __init__(self, name):
        self.name = name
    @classmethod
    def get_apple_to_eve(cls):
        print(cls.snake)

In [9]:
s = Story('anyone')

In [10]:
# get_apple_to_eve 是bound方法，查找顺序是先找s的__dict__是否有get_apple_to_eve,如果没有，查类Story
s.get_apple_to_eve

<bound method Story.get_apple_to_eve of <class '__main__.Story'>>

In [11]:
s.get_apple_to_eve()

python


In [13]:
# 类和实例都可以使用
Story.get_apple_to_eve()

python


In [14]:
s.__dict__

{'name': 'anyone'}

In [19]:
type(s).__dict__['get_apple_to_eve'].__get__(s, type(s))

<bound method Story.get_apple_to_eve of <class '__main__.Story'>>

### @classmehtod起到构造函数的作用，可以这构造前提供预处理的作用

#### V1.0

In [25]:
class Kls2:
    def __init__(self, fname, lname):
        self.fname = fname
        self.lname = lname
    def print_name(self):
        print(f'First Name is {self.fname}.')
        print(f'Last Name is {self.lname}.')

In [26]:
me = Kls2('Yann', 'Li')
me.print_name()

First Name is Yann.
Last Name is Li.


In [27]:
#增加 提前处理的函数,处理这种输入形式'Yann-Li'.
def pre_name(object, name):
    fname, lname = name.split('-')
    return object(fname, lname)

me2 = pre_name(Kls2, 'Yann-Li')
me2.print_name()

First Name is Yann.
Last Name is Li.


#### V2.0

In [30]:
class Kls3:
    def __init__(self, fname, lname):
        self.fname = fname
        self.lname = lname
    def print_name(self):
        print(f'First Name is {self.fname}.')
        print(f'Last Name is {self.lname}.')
        
    @classmethod
    def pre_name(cls, name):
        fname = ''
        lname = ''
        if '-' in name:
            fname, lname = name.split('-')
        elif ',' in name:
            fname, lname = name.split(',')
        elif ' ' in name:
            fname, lname = name.split(' ')
        else:
            fname = name
        return cls(fname, lname)

In [31]:
me3 = Kls3.pre_name('Yann-Li')
me3.print_name()

First Name is Yann.
Last Name is Li.


In [34]:
me4 = Kls3.pre_name('Yann,Li')
me4.print_name()

First Name is Yann.
Last Name is Li.


In [35]:
me5 = Kls3.pre_name('Yann Li')
me5.print_name()

First Name is Yann.
Last Name is Li.


In [36]:
me6 = Kls3.pre_name('YannLi')
me6.print_name()

First Name is YannLi.
Last Name is .


* 类方法用在模拟java定义多个构造函数的情况。
* 由于python类中只能有一个初始化方法，不能按照不同的情况初始化类。

In [37]:
class Book(object):

    def __init__(self, title):
        self.title = title

    @classmethod
    def create(cls, title):
        book = cls(title=title)
        return book

book1 = Book("python")
book2 = Book.create("python and django")
print(book1.title)
print(book2.title)

python
python and django


### 父类定义@classmethod，子类就可以和这个classmethod绑定了

In [46]:
class Fruit:
    total = 0
    
    @classmethod
    def print_total(cls):
        print(cls.total)
        print(id(Fruit.total))
        print(id(cls.total))
    
    @classmethod
    def set(cls, value):
        print(f'Calling {cls}, {value}.')
        cls.total = value

class Apple(Fruit):
    pass

class Orange(Fruit):
    pass

In [47]:
Apple.set(100)

Calling <class '__main__.Apple'>, 100.


In [48]:
Orange.set(200)

Calling <class '__main__.Orange'>, 200.


In [49]:
Apple.print_total()

100
4335932512
4335935712


In [50]:
Orange.print_total()

200
4335932512
4335938912


In [51]:
o = Orange()
o.set(300)

Calling <class '__main__.Orange'>, 300.


In [52]:
o.print_total()

300
4335932512
4416611536


In [53]:
o.__dict__

{}

In [56]:
Orange.print_total()

300
4335932512
4416611536


In [57]:
## 注意静态字段和对象字段的区别：
o.total = 999 ##此时为对象o添加自己的total字段，有别与Orange类的total.

In [58]:
o.__dict__

{'total': 999}

In [59]:
o.print_total()

300
4335932512
4416611536


In [61]:
id(o.total)

4416612016